# Vacuum command

In [39]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,DateType,FloatType,DoubleType

schema1 = StructType([
    StructField('Education_Level',StringType()),
    StructField('Line_Number',IntegerType()),
    StructField('Year',IntegerType()),
    StructField('Month',StringType()),
    StructField('State',StringType()),
    StructField('Labor_Force',IntegerType()),
    StructField('Employed',IntegerType()),
    StructField('Unemployed',IntegerType()),
    StructField('Industry',StringType()),
    StructField('Gender',StringType()),
    StructField('Date_Inserted',StringType()),
    StructField('UnEmployed_Rate_Percentage',DoubleType()),
    StructField('Min_Salary_USD',IntegerType()),
    StructField('Max_Salary_USD',IntegerType()),
    StructField('dense_rank',IntegerType())
])

In [40]:
df = spark.read.format('csv')\
               .option('header','true')\
               .schema(schema1)\
               .load('abfss://delta@projectsynapsestorage.dfs.core.windows.net/Required Files/SchemaManagementDelta.csv')

In [38]:
df.printSchema()

In [41]:
df.createOrReplaceTempView('df_view')

In [42]:
%%sql
-- Lets create a delta table

CREATE TABLE delta.VacTable
USING DELTA
AS
SELECT *
FROM df_view

#### Inserts

In [43]:
%%sql
-- Lets do some INSERTS, UPDATES and DELETES on table

INSERT INTO delta.VacTable
VALUES
    ('Bachelor', 1, 2023, 7, 'NY', 5000, 4500, 500, 'Networking', 'Male', '2023-07-12', 10.5, 40000, 60000, 1),
    ('Master', 2, 2023, 7, 'CA', 7000, 6500, 500, 'Networking', 'Female', '2023-07-12', 8.2, 50000, 80000, 2),
    ('High School', 3, 2023, 7, 'TX', 4000, 3500, 500, 'Networking', 'Male', '2023-07-12', 12.1, 30000, 40000, 3),
    ('PhD', 4, 2023, 7, 'FL', 6000, 5500, 500, 'Networking', 'Female', '2023-07-12', 7.9, 60000, 90000, 4);

##### Updates and deletes

In [44]:
%%sql
-- Perform updates and delete

UPDATE delta.VacTable
SET Education_Level = 'Phd'
WHERE Industry = 'Networking';


DELETE FROM delta.VacTable
WHERE Education_Level = 'Phd';



In [45]:
%%sql

DESCRIBE HISTORY delta.vactable

In [46]:
%%sql

-- This will give list of first 1000 files that will be deleted
-- This will not actually delete it will just show what will be deleted
VACUUM delta.vactable DRY RUN

In [47]:
%%sql 

-- Change retention for less than 7 days using RETAIN

VACUUM delta.vactable RETAIN 5 HOURS

In [48]:
spark.conf.set('spark.databricks.delta.retentionDurationCheck.enabled','False')

In [49]:
%%sql 

-- Change retention for less than 7 days using RETAIN

VACUUM delta.vactable RETAIN 5 HOURS

In [50]:
%%sql

DESCRIBE HISTORY delta.vactable

In [51]:
%%sql 


VACUUM delta.vactable RETAIN 0 HOURS

In [52]:
%%sql

DESCRIBE HISTORY delta.vactable